In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages


[None, None, None, None, None, None, None]

# ATMCS

In [ ]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY)

In [ ]:
await asyncio.sleep(5.)
print("Done")

await asyncio.gather(salobj.set_summary_state(atmcs, salobj.State.ENABLED),
                     salobj.set_summary_state(atptg, salobj.State.ENABLED))

# ATPtg

In [ ]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY)

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=0, elDegs=80.)

In [6]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [7]:
ra = Angle("00 05 22.2352438945", unit=u.hour)
dec = Angle("+03 36 21.894587630", unit=u.deg)
target_name="HD 43"
radec = ICRS(ra, dec)

In [8]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteTelemetry(ATDome, 1, position) falling behind; read 10 messages


58739.20775340555


RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 13 messages


slew...


In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [10]:
await atptg.cmd_offsetAzEl.set_start(el=-8., 
                                     az=35., 
                                         num=0)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 19 messages
falling behind; queue contains 44 elements
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 19 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 19 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 19 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-20., 
                                     az=40., 
                                         num=0)

In [11]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

RemoteTelemetry(ATDome, 1, position) falling behind; read 59 messages
falling behind; queue contains 26 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 12 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue co

In [ ]:
await asyncio.gather(atmcs.close(), 
                     atptg.close(),
                     ataos.close(),
                     atpne.close(),
                     athex.close(),
                     atdome.close(),
                     atdomtraj.close())

In [ ]:
20*360/24., 22*360/24.

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20, u=0.35, v=0.22)

In [ ]:
cam = salobj.Remote(d, "GenericCamera", 1)

In [ ]:
await cam.start_task

In [ ]:
import numpy as np

In [ ]:
z_seq = np.linspace(0.1, 0.35, 6)
print(z_seq)

In [ ]:
z_seq[3:]

In [ ]:
for z in z_seq[3:]:
    print(f"Positioning hexapod @ z={z:4.2f}")
    await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=z, u=0.35, v=0.22)

    print(f"Taking images...")
    await cam.cmd_takeImages.set_start(numImages=3,
                                     expTime=10.,
                                     shutter=True,
                                     imageSequenceName=f'HD193896_z{z:4.2f}_10s'
                                    )

In [ ]:
1*360/24.